In [14]:
from pymongo import MongoClient
import pandas as pd


Livestream
AdaptiveBlue
Ustream
Funny Or Die
sezmi
YouBeQB
Contrastream
Crunchyroll
hulu
MediaPops
ChessPark
alluc
Jango
Qloud
DraftMix
FilmCrave
GotCast
doof
YouLicense
Picktastic
LiveHire
Fancast
Outspark
Zynga
WonderHowTo
Triptuber
JukeFly
Volo Media
Social Gaming Network
Newsclipper
Paprika Lab
HD Greetings
CineGoGo
Redux
Raptr
Camlink
I Heart Movies
Sporting Connections
Taltopia
ScanDigital
SportsUltra
politicsultra
Zooors Interactive Yachting Society
Crootpad
Playfish
Dotsub
Slicethepie
Handipoints
Clipta
Viddyou
XIHA
MyMiniLife
tunesBag
Mytopia
GoCrossCampus
HungryFlix
itbreaks
Episodic
Rosterbot
stickychicken
AudioMicro
Zankapfel
eRepublik
Topspin Media
Myplaylist
Guitarati
Teet
NEHST Studios
Plexonic
YUPPTV
Sutros
imcandy
TiBconcerts
Drop Magazine
MUBI
Modern Feed
Omnivents
FeedFlix
Serious Business
twit+
URockHard
Rekatu
GodTube
Jinni
FantasyBook
RouteNote
Fuzz
Posh24
MeFeedia
Sometrics
Wuapi
YouTomb
PodComics Studio
Open Television Network
Cam-Trax Technologies
snuzu
TriFame

In [21]:
def mongo(sector,year):
 client = MongoClient('localhost', 27017)

 db = client('companies')
 for comp in db.companies.find({"$and":[{"category_code":sector},{'founded_year':{'$gte':year}},{'offices.latitude':{'$exists':True}},{'offices.longitude':{'$exists':True}},{'deadpooled_year':None}]},{'name':1, 'founded_year': 1, '_id': 0, 'number_of_employees':1,
                                 'offices.latitude':1, 'offices.longitude':1, 'funding_rounds.raised_amount':1, 
                                 'funding_rounds.raised_currency_code':1, 'category_code':1, 'total_money_raised':1}):
     print(comp['name'])

- VISUALIZATION PROJECT Geospatial Business Intelligence (BI)
    * Make a geospartial analysis of the `companies` dataset
    * Things you know:
        - You have a software company with 50 employees
        - The company creates video games
        - Roles in your company: 20 developers, 20 Designers/Creatieves/UX/UI and 10 executives/managers
    * Do an analysis about placing the new company offices in the best environment based on the following criteria:
        - There should be software engineers working around
        - The surroundings must have a good ratio of big companies vs startups
        - Ensure you have in your surroundings companies that cover the interests of your team
        - Avoid old companies, prefer recently created ones







In [15]:
comg=db.companies.find({"$and":[{"category_code":"games_video"},{'founded_year':{'$gte':2007}},{'offices.latitude':{'$exists':True}},{'offices.longitude':{'$exists':True}},{'deadpooled_year':None}]},{'name':1, 'founded_year': 1, '_id': 0, 'number_of_employees':1,
                                 'offices.latitude':1, 'offices.longitude':1, 'funding_rounds.raised_amount':1, 
                                 'funding_rounds.raised_currency_code':1, 'category_code':1, 'total_money_raised':1})
co=pd.DataFrame(comg)
co.head()

,category_code,founded_year,funding_rounds,name,number_of_employees,offices,total_money_raised
0,games_video,2007,"[{'raised_amount': 1200000, 'raised_currency_c...",Livestream,120.0,"[{'latitude': 40.726155, 'longitude': -73.9956...",$14.7M
1,games_video,2007,"[{'raised_amount': 1500000, 'raised_currency_c...",AdaptiveBlue,15.0,"[{'latitude': 40.801358, 'longitude': -74.3372}]",$24M
2,games_video,2007,"[{'raised_amount': 1700000, 'raised_currency_c...",Ustream,250.0,"[{'latitude': 37.392936, 'longitude': -122.079...",$60.1M
3,games_video,2007,"[{'raised_amount': None, 'raised_currency_code...",Funny Or Die,NaN,"[{'latitude': 37.421621, 'longitude': -122.137...",$18M
4,games_video,2007,"[{'raised_amount': 17500000, 'raised_currency_...",sezmi,NaN,"[{'latitude': 37.524422, 'longitude': -122.265...",$78.6M


In [16]:
set(co)

{'category_code',
 'founded_year',
 'funding_rounds',
 'name',
 'number_of_employees',
 'offices',
 'total_money_raised'}

In [17]:

def getlat(coor): 
    try: 
        return coor[0]['latitude']
    except:
        return None

   
def getlong(coor): 
    try: 
        return coor[0]['longitude']
    except:
        return None

   
def insert_latlong(comp):   
    comp['latitude'] = comp['offices'].apply(getlat)
    comp['longitude'] = comp['offices'].apply(getlong)
    return comp

In [18]:
co.head()

,category_code,founded_year,funding_rounds,name,number_of_employees,offices,total_money_raised
0,games_video,2007,"[{'raised_amount': 1200000, 'raised_currency_c...",Livestream,120.0,"[{'latitude': 40.726155, 'longitude': -73.9956...",$14.7M
1,games_video,2007,"[{'raised_amount': 1500000, 'raised_currency_c...",AdaptiveBlue,15.0,"[{'latitude': 40.801358, 'longitude': -74.3372}]",$24M
2,games_video,2007,"[{'raised_amount': 1700000, 'raised_currency_c...",Ustream,250.0,"[{'latitude': 37.392936, 'longitude': -122.079...",$60.1M
3,games_video,2007,"[{'raised_amount': None, 'raised_currency_code...",Funny Or Die,NaN,"[{'latitude': 37.421621, 'longitude': -122.137...",$18M
4,games_video,2007,"[{'raised_amount': 17500000, 'raised_currency_...",sezmi,NaN,"[{'latitude': 37.524422, 'longitude': -122.265...",$78.6M
